In [1]:
import json
import os

import numpy as np
from tqdm.auto import tqdm

from verification_system import VerificationSystem

# Experiments without Fine-Tuning

In [2]:
verification_system = VerificationSystem(database_path=os.path.join("data", "database"))

24-04-08 19:58:25 - Searching data\database\incoming_users\authorized_users\1\000023.jpg in 1826 length datastore
24-04-08 19:58:26 - find function duration 1.5776190757751465 seconds


In [2]:
experiment_directories = os.listdir(os.path.join("data", "database", "incoming_users"))
experiment_luminance_directories = list(
    filter(lambda x: "lum" in x, experiment_directories)
)
experiment_psnr_directories = list(
    filter(lambda x: "psnr" in x, experiment_directories)
)

In [4]:
dict_results_luminance = {}
dict_results_psnr = {}

## Luminance

In [ ]:
for index in tqdm(
    iterable=range(int(len(experiment_luminance_directories) / 2)),
    desc="Processing Luminance",
):
    authorized_users_path = os.path.join(
        "data", "database", "incoming_users", experiment_luminance_directories[index]
    )
    unauthorized_users_path = os.path.join(
        "data",
        "database",
        "incoming_users",
        experiment_luminance_directories[
            index + int(len(experiment_luminance_directories) / 2)
        ],
    )
    experiment_dict_key = experiment_luminance_directories[index].replace(
        "_authorized_users", ""
    )

    try:
        df_authorized = verification_system.verify_multiple_users(authorized_users_path)
        df_unauthorized = verification_system.verify_multiple_users(
            unauthorized_users_path
        )

        false_acceptance_rate, false_rejection_rate = (
            verification_system.calculate_far_frr(df_authorized, df_unauthorized)
        )
        true_negative, false_positive, false_negative, true_positive = (
            verification_system.calculate_ROC_curve(
                df_authorized,
                df_unauthorized,
                os.path.join(
                    "data", "results", "roc_curve", experiment_dict_key + ".png"
                ),
            )
        )
        accuracy = (true_positive + true_negative) / (
            true_positive + true_negative + false_positive + false_negative
        )
        access_granted_rate = verification_system.calculate_access_granted_rate(
            df_authorized
        )

        dict_results_luminance[experiment_dict_key] = {
            "FAR": np.round(false_acceptance_rate, 3),
            "FRR": np.round(false_rejection_rate, 3),
            "TN": true_negative,
            "FP": false_positive,
            "FN": false_negative,
            "TP": true_positive,
            "ACC": np.round(accuracy, 3),
            "AGR": np.round(access_granted_rate, 3),
        }
    except Exception as e:
        print(f"Error: {e}")
        dict_results_luminance[experiment_dict_key] = {
            "FAR": -1,
            "FRR": -1,
            "TN": -1,
            "FP": -1,
            "FN": -1,
            "TP": -1,
            "ACC": -1,
            "AGR": -1,
        }
        continue

In [13]:
# convert int64 values to native Python integers
for key, value in dict_results_luminance.items():
    dict_results_luminance[key] = {
        k: int(v) if isinstance(v, np.int64) else v for k, v in value.items()
    }

In [14]:
with open(
    os.path.join("data", "results", "luminance_experiments.json"), "w"
) as json_file:
    json.dump(dict_results_luminance, json_file, indent=4)

## PSNR

In [ ]:
for index in tqdm(
    iterable=range(int(len(experiment_psnr_directories) / 2)),
    desc="Processing PSNR",
):
    authorized_users_path = os.path.join(
        "data", "database", "incoming_users", experiment_psnr_directories[index]
    )
    unauthorized_users_path = os.path.join(
        "data",
        "database",
        "incoming_users",
        experiment_psnr_directories[index + int(len(experiment_psnr_directories) / 2)],
    )
    experiment_dict_key = experiment_psnr_directories[index].replace(
        "_authorized_users", ""
    )

    try:
        df_authorized = verification_system.verify_multiple_users(authorized_users_path)
        df_unauthorized = verification_system.verify_multiple_users(
            unauthorized_users_path
        )

        false_acceptance_rate, false_rejection_rate = (
            verification_system.calculate_far_frr(df_authorized, df_unauthorized)
        )
        true_negative, false_positive, false_negative, true_positive = (
            verification_system.calculate_ROC_curve(
                df_authorized,
                df_unauthorized,
                os.path.join(
                    "data", "results", "roc_curve", experiment_dict_key + ".png"
                ),
            )
        )
        accuracy = (true_positive + true_negative) / (
            true_positive + true_negative + false_positive + false_negative
        )
        access_granted_rate = verification_system.calculate_access_granted_rate(
            df_authorized
        )

        dict_results_psnr[experiment_dict_key] = {
            "FAR": np.round(false_acceptance_rate, 3),
            "FRR": np.round(false_rejection_rate, 3),
            "TN": true_negative,
            "FP": false_positive,
            "FN": false_negative,
            "TP": true_positive,
            "ACC": np.round(accuracy, 3),
            "AGR": np.round(access_granted_rate, 3),
        }
    except Exception as e:
        print(f"Error: {e}")
        dict_results_psnr[experiment_dict_key] = {
            "FAR": -1,
            "FRR": -1,
            "TN": -1,
            "FP": -1,
            "FN": -1,
            "TP": -1,
            "ACC": -1,
            "AGR": -1,
        }
        continue

In [16]:
# convert int64 values to native Python integers
for key, value in dict_results_psnr.items():
    dict_results_psnr[key] = {
        k: int(v) if isinstance(v, np.int64) else v for k, v in value.items()
    }

In [17]:
with open(os.path.join("data", "results", "psnr_experiments.json"), "w") as json_file:
    json.dump(dict_results_psnr, json_file, indent=4)

# Experiments with Fine-Tuning

## Luminance Linear 1.5

In [12]:
verification_system = VerificationSystem(
    database_path=os.path.join("data", "database"),
    destination="lum_fine_authorized_users_linear_1.5",
)

24-04-08 21:31:58 - Searching data\database\incoming_users\authorized_users\1\000023.jpg in 3653 length datastore
24-04-08 21:31:58 - find function duration 0.38701438903808594 seconds


In [13]:
dict_results_fine_luminance_linear = {}

In [ ]:
for index in tqdm(
    iterable=range(int(len(experiment_luminance_directories) / 2)),
    desc="Processing Luminance",
):
    authorized_users_path = os.path.join(
        "data", "database", "incoming_users", experiment_luminance_directories[index]
    )
    unauthorized_users_path = os.path.join(
        "data",
        "database",
        "incoming_users",
        experiment_luminance_directories[
            index + int(len(experiment_luminance_directories) / 2)
        ],
    )
    experiment_dict_key = experiment_luminance_directories[index].replace(
        "_authorized_users", ""
    )

    try:
        df_authorized = verification_system.verify_multiple_users(authorized_users_path)
        df_unauthorized = verification_system.verify_multiple_users(
            unauthorized_users_path
        )

        false_acceptance_rate, false_rejection_rate = (
            verification_system.calculate_far_frr(df_authorized, df_unauthorized)
        )
        true_negative, false_positive, false_negative, true_positive = (
            verification_system.calculate_ROC_curve(
                df_authorized,
                df_unauthorized,
                os.path.join(
                    "data",
                    "results",
                    "roc_curve_fine_luminance_linear",
                    experiment_dict_key + ".png",
                ),
            )
        )
        accuracy = (true_positive + true_negative) / (
            true_positive + true_negative + false_positive + false_negative
        )
        access_granted_rate = verification_system.calculate_access_granted_rate(
            df_authorized
        )

        dict_results_fine_luminance_linear[experiment_dict_key] = {
            "FAR": np.round(false_acceptance_rate, 3),
            "FRR": np.round(false_rejection_rate, 3),
            "TN": true_negative,
            "FP": false_positive,
            "FN": false_negative,
            "TP": true_positive,
            "ACC": np.round(accuracy, 3),
            "AGR": np.round(access_granted_rate, 3),
        }
    except Exception as e:
        print(f"Error: {e}")
        dict_results_fine_luminance_linear[experiment_dict_key] = {
            "FAR": -1,
            "FRR": -1,
            "TN": -1,
            "FP": -1,
            "FN": -1,
            "TP": -1,
            "ACC": -1,
            "AGR": -1,
        }
        continue

In [15]:
# convert int64 values to native Python integers
for key, value in dict_results_fine_luminance_linear.items():
    dict_results_fine_luminance_linear[key] = {
        k: int(v) if isinstance(v, np.int64) else v for k, v in value.items()
    }

In [16]:
with open(
    os.path.join("data", "results", "fine_luminance_linear_experiments.json"), "w"
) as json_file:
    json.dump(dict_results_fine_luminance_linear, json_file, indent=4)

## PSNR 10

In [3]:
verification_system = VerificationSystem(
    database_path=os.path.join("data", "database"),
    destination="gauss_fine_authorized_users_psnr_10",
)

24-04-08 22:13:45 - Searching data\database\incoming_users\authorized_users\1\000023.jpg in 3645 length datastore
24-04-08 22:13:46 - find function duration 1.7300794124603271 seconds


In [4]:
dict_results_fine_psnr_10 = {}

In [ ]:
for index in tqdm(
    iterable=range(int(len(experiment_psnr_directories) / 2)),
    desc="Processing PSNR",
):
    authorized_users_path = os.path.join(
        "data", "database", "incoming_users", experiment_psnr_directories[index]
    )
    unauthorized_users_path = os.path.join(
        "data",
        "database",
        "incoming_users",
        experiment_psnr_directories[index + int(len(experiment_psnr_directories) / 2)],
    )
    experiment_dict_key = experiment_psnr_directories[index].replace(
        "_authorized_users", ""
    )

    try:
        df_authorized = verification_system.verify_multiple_users(authorized_users_path)
        df_unauthorized = verification_system.verify_multiple_users(
            unauthorized_users_path
        )

        false_acceptance_rate, false_rejection_rate = (
            verification_system.calculate_far_frr(df_authorized, df_unauthorized)
        )
        true_negative, false_positive, false_negative, true_positive = (
            verification_system.calculate_ROC_curve(
                df_authorized,
                df_unauthorized,
                os.path.join(
                    "data",
                    "results",
                    "roc_curve_fine_psnr_10",
                    experiment_dict_key + ".png",
                ),
            )
        )
        accuracy = (true_positive + true_negative) / (
            true_positive + true_negative + false_positive + false_negative
        )
        access_granted_rate = verification_system.calculate_access_granted_rate(
            df_authorized
        )

        dict_results_fine_psnr_10[experiment_dict_key] = {
            "FAR": np.round(false_acceptance_rate, 3),
            "FRR": np.round(false_rejection_rate, 3),
            "TN": true_negative,
            "FP": false_positive,
            "FN": false_negative,
            "TP": true_positive,
            "ACC": np.round(accuracy, 3),
            "AGR": np.round(access_granted_rate, 3),
        }
    except Exception as e:
        print(f"Error: {e}")
        dict_results_fine_psnr_10[experiment_dict_key] = {
            "FAR": -1,
            "FRR": -1,
            "TN": -1,
            "FP": -1,
            "FN": -1,
            "TP": -1,
            "ACC": -1,
            "AGR": -1,
        }
        continue

In [6]:
# convert int64 values to native Python integers
for key, value in dict_results_fine_psnr_10.items():
    dict_results_fine_psnr_10[key] = {
        k: int(v) if isinstance(v, np.int64) else v for k, v in value.items()
    }

In [7]:
with open(
    os.path.join("data", "results", "fine_psnr_10_experiments.json"), "w"
) as json_file:
    json.dump(dict_results_fine_psnr_10, json_file, indent=4)